In [3]:
import pandas as pd

horse_df = pd.read_csv("horse_data.csv",
                   na_values="?",
                   usecols=[0,1,3,4,8,12,17,22],
                   names=["surgery", "age", "rectal_temp", "pulse", "muc_membran", "abd_distension","abdomen","outcome"])
modes={col:horse_df[col].round().mode()[0] for col in horse_df.columns}
horse_df_desc = horse_df.describe()
m_series = pd.Series(data=modes, name="mode")
horse_df_desc = horse_df_desc.append(m_series, ignore_index=False)
print(horse_df_desc)
'''
Более чем в 50% и до 75% случаях делали операции
Более 75% старше 6 месяцев
Средняя температура ~ 38.2
Наиболее частый пульс 48, средний 72 за счёт высоких показателе в отдельных случаях
Исходя из цвета слизистой  в ~50% не было серьёзных нарушений, где-то 25% начальный шок и 25% имело серьёзные проблемы 
включая сепсис 
Более чем у половины не было серьёзного вздутия живота где-то от 25% имели такую проблему
Примерно в 50% случаев были проблемы с толстым и тонким кишечником требующие хирургического вмешательства
Более 50% лошадей выжали, примерно 25% погибли часть из них вероятно из-за сепсиса
'''

          surgery         age  rectal_temp       pulse  muc_membran  \
count  299.000000  300.000000   240.000000  276.000000   253.000000   
mean     1.397993    1.640000    38.167917   71.913043     2.853755   
std      0.490305    2.173972     0.732289   28.630557     1.620294   
min      1.000000    1.000000    35.400000   30.000000     1.000000   
25%      1.000000    1.000000    37.800000   48.000000     1.000000   
50%      1.000000    1.000000    38.200000   64.000000     3.000000   
75%      2.000000    1.000000    38.500000   88.000000     4.000000   
max      2.000000    9.000000    40.800000  184.000000     6.000000   
mode     1.000000    1.000000    38.000000   48.000000     1.000000   

       abd_distension     abdomen     outcome  
count      244.000000  182.000000  299.000000  
mean         2.266393    3.692308    1.551839  
std          1.065131    1.491725    0.737187  
min          1.000000    1.000000    1.000000  
25%          1.000000    2.000000    1.000000  
5

'\nБолее чем в 50% и до 75% случаях делали операции\nБолее 75% старше 6 месяцев\nСредняя температура ~ 38.2\nНаиболее частый пульс 48, средний 72 за счёт высоких показателе в отдельных случаях\nИсходя из цвета слизистой  в ~50% не было серьёзных нарушений, где-то 25% начальный шок и 25% имело серьёзные проблемы включая сепсис \nБолее чем у половины не было серьёзного вздутия живота где-то от 25% имели такую проблему\nПримерно в 50% случаев были проблемы с толстым и тонким кишечником требующие хирургического вмешательства\nБолее 50% лошадей выжали, примерно 25% погибли часть из них вероятно из-за сепсиса\n'

In [4]:
q1 = horse_df["rectal_temp"].quantile(0.25)
q3 = horse_df["rectal_temp"].quantile(0.75)
iqr = q3 - q1
lower_bound_tempr = q1 - (1.5 * iqr) 
upper_bound_tempr = q3 + (1.5 * iqr)
remove_outliers = horse_df[horse_df["rectal_temp"].between(lower_bound_tempr, upper_bound_tempr, inclusive=True)]
print(pd.concat([horse_df, remove_outliers]).drop_duplicates(keep=False)[horse_df["rectal_temp"].notna()])


q1 = horse_df["pulse"].quantile(0.25)
q3 = horse_df["pulse"].quantile(0.75)
iqr = q3 - q1
lower_bound_pulse = q1 - (1.5 * iqr) 
upper_bound_pulse = q3 + (1.5 * iqr)
remove_outliers = horse_df[horse_df["pulse"].between(lower_bound_pulse, upper_bound_pulse, inclusive=True)]
print(pd.concat([horse_df, remove_outliers]).drop_duplicates(keep=False)[horse_df["pulse"].notna()])
'''
выбросы по  температуре и пульсу в пределах допустимых значений, я бы оставил всё как есть.
Разве что в строки 259 у лошади температура 40.8   при относительно нормальных показателях по другим параметрам,
хотя пульс повышен. Тут или не хорактерное заболевание для диагностики которого надо ввести дополнительные параметры,
либо строку с этим выбросом стоит удалить.

'''


     surgery  age  rectal_temp  pulse  muc_membran  abd_distension  abdomen  \
20       1.0    1         39.9   72.0          5.0             4.0      4.0   
44       1.0    1         35.4  140.0          4.0             NaN      5.0   
54       2.0    1         40.3  114.0          1.0             3.0      5.0   
75       1.0    9         39.7  100.0          5.0             NaN      NaN   
80       1.0    1         36.4   98.0          4.0             2.0      4.0   
91       2.0    1         40.3  114.0          1.0             3.0      5.0   
99       2.0    1         39.6  108.0          6.0             3.0      5.0   
118      1.0    1         36.5   78.0          1.0             1.0      NaN   
141      2.0    1         36.0   42.0          5.0             NaN      NaN   
238      2.0    1         36.1   88.0          3.0             2.0      4.0   
251      2.0    1         36.6   42.0          2.0             1.0      5.0   
259      1.0    1         40.8   72.0          1.0  

<ipython-input-4-5e458212ef04>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(pd.concat([horse_df, remove_outliers]).drop_duplicates(keep=False)[horse_df["rectal_temp"].notna()])
<ipython-input-4-5e458212ef04>:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(pd.concat([horse_df, remove_outliers]).drop_duplicates(keep=False)[horse_df["pulse"].notna()])


'\nвыбросы по  температуре и пульсу в пределах допустимых значений, я бы оставил всё как есть.\nРазве что в строки 259 у лошади температура 40.8   при относительно нормальных показателях по другим параметрам,\nхотя пульс повышен. Тут или не хорактерное заболевание для диагностики которого надо ввести дополнительные параметры,\nлибо строку с этим выбросом стоит удалить.\n\n'

In [5]:
for col in horse_df.columns:
    pct_missing = horse_df[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

surgery - 0.3%
age - 0.0%
rectal_temp - 20.0%
pulse - 8.0%
muc_membran - 15.7%
abd_distension - 18.7%
abdomen - 39.3%
outcome - 0.3%


In [35]:
# fill_median.Age.fillna(titanic.Age.median(), inplace=True)
'''
Замена предыдущим или последующим значением  не релевантна так как каждая строка описывает случай не зависящий от остальных
Буду востанавливать данные с помощью последовательной групировки по разным связанным столбцам, так как не всегда 
получается заполнить все пропуске с первого раза. Возможно группировка по нескольким столбцам одновременно даёт более точный результат
но и оставляет большее количество пробелов, а в случае моды не позволяет вычислить этот показатель на данном наборе данных

surgery - категориальная величина часто зависящая от состояния желудка, буду использовать моду и группировку по столбцам 
описывающих состояние желудка

rectal_temp - числовая величина буду использовать медиану как более устойчивую к выбросам, есть зависимость от пульса и 
цвета слизистой как показателя отображающего общее состояние, плюс возраст

pulse - числовая величина описывающая общее состояние. Буду использовать медиану и группировку по температуре и состоянию 
слизистой, так как эти величины так же описывают общее состояние, плюс возраст 

muc_membran - категориальная величина, мода. Описывает общее состояния лошади,  возможно корелируется с outcome

abd_distension категориальная величина, мода. Думаю корелируется с цветом слизистой.

abdomen - категориальная величина, мода. Коррелирует со степенью вздутия желудка и думаю с цветом слизистой.

outcome -  категориальная величина, мода. Думаю есть зависимость от возраста.  Возможно было бы лучше групировать по 
наличию операуии, но такая груптровка не позволяет вычислить моду

'''
horse_df = pd.read_csv("horse_data.csv",
                   na_values="?",
                   usecols=[0,1,3,4,8,12,17,22],
                   names=["surgery", "age", "rectal_temp", "pulse", "muc_membran", "abd_distension","abdomen","outcome"])

print(horse_df.describe())

#удалить строки у которых заполненно 3 значения из 8
horse_df = horse_df.dropna(thresh=3)

horse_df.outcome.fillna(horse_df.groupby(["age"]).outcome.transform(lambda x: x.mode()[0]), inplace=True)
horse_df.surgery.fillna(horse_df.groupby(['outcome']).surgery.transform(lambda x: x.mode()[0]), inplace=True)

horse_df.rectal_temp.fillna(horse_df.groupby(['pulse']).rectal_temp.transform('median'), inplace=True)
horse_df.rectal_temp.fillna(horse_df.groupby(['muc_membran']).rectal_temp.transform('median'), inplace=True)
horse_df.rectal_temp.fillna(horse_df.groupby(['age']).rectal_temp.transform('median'), inplace=True)

horse_df.pulse.fillna(horse_df.groupby(['rectal_temp']).pulse.transform('median'), inplace=True)
horse_df.pulse.fillna(horse_df.groupby(["muc_membran"]).pulse.transform('median'), inplace=True)
horse_df.pulse.fillna(horse_df.groupby(['age']).pulse.transform('median'), inplace=True)

horse_df.muc_membran.fillna(horse_df.groupby(["outcome"]).muc_membran.transform(lambda x: x.mode()[0]), inplace=True)
horse_df.abd_distension.fillna(horse_df.groupby(["muc_membran"]).abd_distension.transform(lambda x: x.mode()[0]), inplace=True)
horse_df.abdomen.fillna(horse_df.groupby(["abd_distension"]).abdomen.transform(lambda x: x.mode()[0]), inplace=True)


for col in horse_df.columns:
    pct_missing = horse_df[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')
    
horse_df.describe()
'''
ну вроде не плохо получилось, основные показатели изменились не значительно, даже средне квадратическое откланение немного
уменьшилось
'''

          surgery         age  rectal_temp       pulse  muc_membran  \
count  299.000000  300.000000   240.000000  276.000000   253.000000   
mean     1.397993    1.640000    38.167917   71.913043     2.853755   
std      0.490305    2.173972     0.732289   28.630557     1.620294   
min      1.000000    1.000000    35.400000   30.000000     1.000000   
25%      1.000000    1.000000    37.800000   48.000000     1.000000   
50%      1.000000    1.000000    38.200000   64.000000     3.000000   
75%      2.000000    1.000000    38.500000   88.000000     4.000000   
max      2.000000    9.000000    40.800000  184.000000     6.000000   

       abd_distension     abdomen     outcome  
count      244.000000  182.000000  299.000000  
mean         2.266393    3.692308    1.551839  
std          1.065131    1.491725    0.737187  
min          1.000000    1.000000    1.000000  
25%          1.000000    2.000000    1.000000  
50%          2.000000    4.000000    1.000000  
75%          3.000000   

,surgery,age,rectal_temp,pulse,muc_membran,abd_distension,abdomen,outcome
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,1.396667,1.640000,38.177417,71.893333,2.716667,2.163333,3.410000,1.550000
std,0.490023,2.173972,0.665364,27.997765,1.597361,1.058485,1.666681,0.736642
min,1.000000,1.000000,35.400000,30.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.875000,48.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,38.150000,64.000000,3.000000,2.000000,4.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.000000,3.000000,5.000000,2.000000
max,2.000000,9.000000,40.800000,184.000000,6.000000,4.000000,5.000000,3.000000
